In [ ]:
from datetime import datetime
import numpy as np
if '..' not in sys.path:
    sys.path.append('..')
from dlml.utils import collect_experiments

In [ ]:
area_ID = 1
level = 1
time_interval = datetime(2023, 4, 24), datetime(2023, 4, 27)
expts_CNN = collect_experiments(area_ID, area_measure='momentum',
                                rec_bus_IDs=[3],
                                additional_tags=['neural_network',
                                                 'converted_from_PowerFactory',
                                                 'compensators_in_training',
                                                 '1D_pipeline', 'trial_dur_60',
                                                 'data_subset', 'Vd'],
                                time_interval=time_interval,
                                verbose_level=level)
expts_sklearn = collect_experiments(area_ID, area_measure='momentum',
                                    additional_tags=['sklearn'],
                                    verbose_level=level)

In [ ]:
models = 'CNN','SVR','MLP','nearest_neighbors','kernel_ridge','random_forest'
MAPE = {key: [] for key in models}
for ID,expt in expts_sklearn.items():
    for key in MAPE:
        if key in expt['tags']:
            break
    MAPE[key].append(expt['MAPE'])
for ID,expt in expts_CNN.items():
    MAPE['CNN'].append(expt['MAPE'])

In [ ]:
for key in models:
    if len(MAPE[key]) == 1:
        print(key.replace('_',' ') + ' ' + '.' * (25-len(key)) +
              ' {:6.3f}%'.format(MAPE[key][0]))
    else:
        print(key.replace('_',' ') + ' ' + '.' * (25-len(key)) +
              ' {:6.3f}% +- {:5.3f}%'.format(np.mean(MAPE[key]),
                                            np.std(MAPE[key])/np.sqrt(len(MAPE[key]))))

## Table for the paper discussion

In [ ]:
with open('MAPE_comparison_table.tex', 'w') as fid:
    fid.write('\\begin{table}[tb!]\n')
    fid.write('\\begin{center}\n')
    fid.write('\\caption{MAPE\n')
    fid.write('\\label{tab:map}}\n')
    fid.write('\\begin{tabular}{lcc}\n')
    fid.write('\\toprule\n')
    fid.write('\\thead{Model} & \\thead{\\ac{mape} [\\%]} & \\thead{N} \\\\\n')
    fid.write('\\midrule\n')
    for key in models:
        if '_' in key:
            model = key.replace('_',' ').capitalize()
        else:
            model = key
        fid.write(model + ' & ')
        if len(MAPE[key]) == 1:
            fid.write('{:.2f} & 1 \\\\\n'.format(MAPE[key][0]))
        else:
            fid.write('{:.2f} $\\pm$ {:.2f} & {} \\\\\n'.
                      format(np.mean(MAPE[key]),
                             np.std(MAPE[key]) / np.sqrt(len(MAPE[key])),
                             len(MAPE[key])))
    fid.write('\\bottomrule\n')
    fid.write('\\end{tabular}\n')
    fid.write('\\end{center}\n')
    fid.write('\\end{table}\n')